In [1]:
import torch
import argparse
import os
import numpy as np

# TensorBoard
from torch.utils.tensorboard import SummaryWriter

from data import get_dataset
from experiment import ex
from model import load_model

from utils import post_config_hook
from utils.yaml_config_hook import yaml_config_hook
from utils.eval import eval_all
from utils.youtube import download_yt
from datasets.utils.resample import convert_samplerate

import matplotlib.pyplot as plt


cfg = yaml_config_hook("./config/config.yaml")

args = argparse.Namespace(**cfg)
args.dataset = "magnatagatune"
args.model_path = "/storage/jspijkervet/logs_backup_ws7/clmr/2/"
args.epoch_num= 1490
args.finetune_model_path = "/storage/jspijkervet/logs_backup_ws7/clmr/4"
args.finetune_epoch_num = 50
args.n_classes = 50 # 50 tags

/home/jspijkervet/miniconda3/envs/clmr/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/jspijkervet/miniconda3/envs/clmr/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
args.lin_eval = True
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.batch_size = args.logistic_batch_size

In [3]:
(train_loader, train_dataset, val_loader, val_dataset, test_loader, test_dataset) = get_dataset(args)

### Linear / Supervised training, using segmented dataset ###
Num segments: 187060
Num tracks: 18706
/storage/jspijkervet/magnatagatune/statistics_segments_22050.csv
[Train dataset (magnatagatune_segments_22050)]: Loaded mean/std: -7.70555e-05, 0.17842068
### Linear / Supervised training, using segmented dataset ###
Num segments: 18250
Num tracks: 1825
/storage/jspijkervet/magnatagatune/statistics_segments_22050.csv
[Validation dataset (magnatagatune_segments_22050)]: Loaded mean/std: -7.70555e-05, 0.17842068
### Linear / Supervised training, using segmented dataset ###
Num segments: 53290
Num tracks: 5329
/storage/jspijkervet/magnatagatune/statistics_segments_22050.csv
[Test dataset (magnatagatune_segments_22050)]: Loaded mean/std: -7.70555e-05, 0.17842068


In [8]:
context_model, _, _ = load_model(args, reload_model=True, name=args.model_name)
context_model.eval()

args.n_features = context_model.n_features

model, _, _ = load_model(args, reload_model=True, name="supervised")
model = model.to(args.device)

print(context_model)
print(model)

### SampleCNN59049 ###
### RELOADING CLMR MODEL FROM CHECKPOINT 1490 ###
### Using Adam optimizer ###
### RELOADING CLMR OPTIMIZER FROM CHECKPOINT 1490 ###
### RELOADING SUPERVISED MODEL FROM CHECKPOINT 50 ###
### Using Adam optimizer ###
### RELOADING SUPERVISED OPTIMIZER FROM CHECKPOINT 50 ###
CLMR(
  (encoder): SampleCNN59049(
    (conv1): Sequential(
      (0): Conv1d(1, 128, kernel_size=(3,), stride=(3,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
    (conv3): Sequential(
      (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_

In [9]:
args.current_epoch = 0

with torch.no_grad():
    taggram = []
    for idx, x in enumerate(chunks):
        x = x.to(args.device)

        # normalise
        if train_dataset.mean:
            x = train_dataset.normalise_audio(x)

        # add batch dim
        x = x.unsqueeze(0)         
        h, z = context_model(x)

        output = model(h)
        output = torch.nn.functional.softmax(output, dim=1)
        output = output.squeeze(0) # remove batch dim
        taggram.append(output.cpu().detach().numpy())